# Data Cleaning

## Import

In [ ]:
import os
import pathlib
import matplotlib

import numpy as np
import matplotlib.pyplot as plt
import mne
import mne_bids
from mne.preprocessing import (ICA, create_eog_epochs, 
                               compute_proj_eog, create_ecg_epochs, corrmap)
                               
matplotlib.use('Qt5Agg')
mne.set_log_level('warning') #All mne functions will only spit out output if they have reason to.

## Insert Subject Information

In [ ]:
intername = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_interpolated_data.fif'
oldepochs = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_epochs-epo.fif'
filename = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_cleaned_data.fif'
filename

## Input Raw Data

#### Subject One Template

In [ ]:
template_raw = mne.io.read_raw(pathlib.Path('out_data') / '02_interpolation_data' / 
                      'sub-01_ses-01_task-WM1isi0_interpolated_data.fif')
#events, event_id = mne.events_from_annotations(template_raw) 
template_raw.load_data()

#### Actual Data

In [ ]:
raw = mne.io.read_raw(pathlib.Path('out_data') / '02_interpolation_data' / 
                      intername)
events, event_id = mne.events_from_annotations(raw) 
raw.load_data()

## ICA Analysis

#### Template Creation

In [ ]:
high_pass_template = template_raw.filter(l_freq=1, h_freq=None) 

In [ ]:
template_ica = ICA(n_components=15, max_iter='auto', random_state=97)
template_ica.fit(high_pass_template)
template_ica

### Apply a High Pass Filter

In [ ]:
high_pass_raw = raw.filter(l_freq=1, h_freq=None) 

### Run ICA

In [ ]:
ica = ICA(n_components=15, max_iter='auto', random_state=97)
ica.fit(high_pass_raw)
ica

### Evaluate ICA Components

### Reject Eye Blinks Component

In [ ]:
icas = list()
icas.append(template_ica)
icas.append(ica)

In [ ]:
#ica = icas[0]
eog_inds = ica.find_bads_eog(raw)

In [ ]:
corrmap(icas, template=(0, 6), threshold = .9, label = 'eyeblink', plot = False)

In [ ]:
icas[1].exclude = icas[1].labels_['eyeblink']

### Create New Raw Data

In [ ]:
reconst_raw = raw.copy()
icas[1].apply(reconst_raw)

### Evaluate ICA Component Removal

## Signal Space Projection

### Filter Raw Data

In [ ]:
filter_raw = reconst_raw.filter(l_freq=0.1, h_freq=40) 
filter_raw

### Compute EOG Projector 

### Visualize Scalp Distribution

### Apply Signal Space Projectors

## Epoch Comparison

### Input Old Epochs

In [ ]:
old_epochs = mne.read_epochs(pathlib.Path('out_data')
            / '03_epochs' / oldepochs)
old_epochs

#### Rejection Filter

In [ ]:
reject_criteria = dict(eeg=100e-6) # 100 µV, what should our rejection criteria be? 

old_epochs.drop_bad(reject=reject_criteria)

### Run New Epochs

In [ ]:
tmin = -0.25 #300 ms before the event
tmax = 0.8 #500 ms after the onset

new_epochs = mne.Epochs(reconst_raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    preload=True)
new_epochs

In [ ]:
reject_criteria = dict(eeg=100e-6) # 100 µV, what should our rejection criteria be? 

new_epochs.drop_bad(reject=reject_criteria)

## Result

## Saving Cleaned Epoch Data 

In [ ]:
new_epochs.save(pathlib.Path('out_data')
            / '05_cleaned_data' / filename)